## Perceptron

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

class Perceptron(nn.Module):
    def __init__(self, input_size, output_size):
        super(Perceptron, self).__init__()
        self.W = nn.Parameter(torch.randn(output_size, input_size) * 0.01)
        self.b = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        x = torch.matmul(x, self.W.T) + self.b
        return x

## Multi-layer perceptron

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases for the first Linear layer
        self.W_ih = nn.Parameter(torch.randn(hidden_size, input_size) * 0.01)
        self.b_ih = nn.Parameter(torch.zeros(hidden_size))

        # Initialize weights and biases for the second Linear layer
        self.W_ho = nn.Parameter(torch.randn(output_size, hidden_size) * 0.01)
        self.b_ho = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        # Apply the first linear layer with ReLU activation
        h = torch.relu(torch.matmul(x, self.W_ih.T) + self.b_ih)

        # Apply the second linear layer
        x = torch.matmul(h, self.W_ho.T) + self.b_ho
        return x


In [ ]:
def generate_linear_data(N=100, D=2):
    X = np.random.randn(N, D)
    X[:N//2, :] += 1
    X[N//2:, :] -= 1
    Y = np.concatenate((np.zeros(N//2), np.ones(N//2)))
    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y, dtype=torch.long)
    return X, Y

def generate_xor_data():
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
    Y = np.array([0, 1, 1, 0], dtype=np.long)
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

In [ ]:
def train_model(model, criterion, optimizer, X, Y, num_epochs=1000, print_interval=100):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, Y.float())
        loss.backward()
        optimizer.step()
        if (epoch + 1) % print_interval == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, X, Y):
    with torch.no_grad():
        outputs = model(X)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        accuracy = (predicted == Y).sum().item() / Y.size(0)
        print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
def plot_decision_boundary(model, X, Y, title):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
    with torch.no_grad():
        Z = model(grid)
        Z = torch.sigmoid(Z.squeeze())
        Z = (Z > 0.5).numpy()
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', marker='o')
    plt.title(title)
    plt.show()


In [ ]:
# Task 1: Linearly Separate Two Clouds of Dots
X_linear, Y_linear = generate_linear_data()
input_size = 2
output_size = 1

# Perceptron
model_perceptron = Perceptron(input_size, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model_perceptron.parameters(), lr=0.01)
train_model(model_perceptron, criterion, optimizer, X_linear, Y_linear, num_epochs=1000, print_interval=100)
plot_decision_boundary(model_perceptron, X_linear, Y_linear, 'Perceptron - Linear Data')

# MLP
hidden_size = 2
model_mlp = MLP(input_size, hidden_size, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model_mlp.parameters(), lr=0.01)
train_model(model_mlp, criterion, optimizer, X_linear, Y_linear, num_epochs=1000, print_interval=100)
plot_decision_boundary(model_mlp, X_linear, Y_linear, 'MLP - Linear Data')

# Task 2: XOR Task
X_xor, Y_xor = generate_xor_data()

# Perceptron
model_perceptron = Perceptron(input_size, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model_perceptron.parameters(), lr=0.01)
train_model(model_perceptron, criterion, optimizer, X_xor, Y_xor, num_epochs=10000, print_interval=1000)
plot_decision_boundary(model_perceptron, X_xor, Y_xor, 'Perceptron - XOR Data')

# MLP
model_mlp = MLP(input_size, hidden_size, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model_mlp.parameters(), lr=0.1)
train_model(model_mlp, criterion, optimizer, X_xor, Y_xor, num_epochs=10000, print_interval=1000)
plot_decision_boundary(model_mlp, X_xor, Y_xor, 'MLP - XOR Data')